In [ ]:
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Flatten, Dense, MaxPooling2D,Dropout, Conv2D, BatchNormalization,Add,Activation,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [ ]:
train_df=pd.read_csv('../input/minst-fashion-dataset/fashion-mnist_train.csv')
test_df=pd.read_csv('../input/minst-fashion-dataset/fashion-mnist_test.csv')

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head()

One column for label. Rest of the columns should represent both height and width.

In [ ]:
print('Image Pixels: ', 785-1)
print('Lenght of each dimension: ', np.sqrt(784))

Therefore our dimensions should be (28,28,1).
(1 represents grayscale)

In [ ]:
X=train_df.iloc[:,1:]
y=train_df.label
X_test=test_df.iloc[:,1:]
y_test=test_df.label

In [ ]:
X.head(3)

# Image from pixels

### Procedure:
Select values of each row separately and reshape them as (28,28,1). 

In [ ]:
import matplotlib.pyplot as plt
def pic_show(df,row):
    plt.figure()
    plt.imshow(df.iloc[row].values.reshape(28,28,1))
    plt.axis('off')
    plt.show()

pic_show(X,2)

# Defining Reshaper Function

### Procedure:
Select values of each row separately by utilizing for loop and reshape them as (28,28,1). Finally collect modified data in a new list and return the data collection. 

In [ ]:
def reshaper(df):
    X=[]
    for i in range(df.shape[0]):
        pic=df.iloc[i].values.reshape(28,28,1)
        X.append(pic)
    return np.array(X)

        

In [ ]:
X_test

In [ ]:
X=reshaper(X)
X_test=reshaper(X_test)

# Normalization

### Procedure:
Divide data to 255.0 to both converting the result a float and reduce the complexity of data. Normalization is a process in which the ratio between data remains same but the magnitude is decreased (Between 0 and 1). This is done by dividing all data to max value in dataset.

For example: [20,10]

Normalized version:[1,0.5]

For colorscale, the maximum value is 255. That's why we divide all data to 255.0 

In [ ]:
X=X/255.0
X_test=X_test/255.0

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X,y, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_one=to_categorical(y_train)
y_val_one=to_categorical(y_val)
y_test_one=to_categorical(y_test)

# CNN Sequential Model
Important Notes:

For the starting point, 0.5 is generally good for  Dropout layer (True for Linear Neural Networks)(Claim). Although that's the case it's beneficial to try the other alternatives until finding the optimum value. 

Let's test it!

### Precedure
Try different dropout values in model with utilizg for loop.

Use lower epoch for the convenience.




In [ ]:
from tensorflow.keras.optimizers import SGD
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)


In [ ]:

             
candidates=[0.1,0.3,0.5,0.7,0.9]

for i in candidates:
    model=Sequential([
        Conv2D(64,3,activation='relu', input_shape=(28,28,1)),
        BatchNormalization(),
        MaxPooling2D(2),
        Dropout(i),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy',
                 metrics=['accuracy'],
                 optimizer=sgd)
    history=model.fit(X_train,y_train_one, batch_size=128,epochs=5,validation_data=(X_val,y_val_one), verbose=0)
    print('Test accuracy for dropout value: ', str(i), model.evaluate(X_test,y_test_one,verbose=0)[1])

In [ ]:
import seaborn as sns
plt.ylabel('Test Accuracy')
plt.xlabel('Dropout Value')
sns.barplot(x=[0.1,0.3,0.5,0.7,0.9],y=[0.915,0.912,0.912,0.90,0.87]);

It seems like except dropout value of 0.9, test accuracy doesn't change that much. 

Let's try higher epoch and compare it with a functional neural network.

In [ ]:
model=Sequential([
    Conv2D(64,3,activation='relu', input_shape=(28,28,1)),
    BatchNormalization(),
    MaxPooling2D(2),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(loss='categorical_crossentropy',
             metrics=['accuracy'],
             optimizer=sgd)
history=model.fit(X_train,y_train_one, batch_size=128,epochs=20,validation_data=(X_val,y_val_one), verbose=0)

In [ ]:
history_df=pd.DataFrame(history.history)
history_df[['loss','val_loss']].plot();

In [ ]:
print('Test Accuracy for Sequential Neural Network:', model.evaluate(X_test,y_test_one,verbose=0)[1])

# CNN Functional Model

### Procedure:

Add another Dense Layer after flatten layer, combine the outputs and run the functional model.

Finally compare the accuracy between two!

In [ ]:
input_tensor=Input(shape=(28,28,1))
Conv1=Conv2D(64,3, input_shape=(28,28,1))(input_tensor)
A1=Activation('relu')(Conv1)
BatchNor=BatchNormalization()(A1)
Pool1=MaxPooling2D(2)(BatchNor)
Pool1=Dropout(0.5)(Pool1)
Flat=Flatten()(Pool1)
D1=Dense(128, activation='relu')(Flat)
D2=Dense(128, activation='relu')(Flat)
D3=Add()([D2,D1])
output_tensor=Dense(10,activation='softmax')(D3)
model=Model(inputs=input_tensor,outputs=output_tensor)

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(model)

In [ ]:
model.compile(loss='categorical_crossentropy',
             metrics=['accuracy'],
             optimizer=sgd)
history=model.fit(X_train,y_train_one, batch_size=128,epochs=20,validation_data=(X_val,y_val_one), verbose=0)

In [ ]:
print('Test Accuracy for Functional Neural Network:', model.evaluate(X_test,y_test_one,verbose=0)[1])

# Conclusion

Making more complex neural network models may improve the performance of the model. This improvement depends on the complexity that is added.

Dropout values generally did not change the accuracy of the model that much. 0.5 can be a good starting point as mentioned in the claim.
